# Linjär Regressionsanalys av Bostadspriser i Kalifornien

## Introduktion

I denna laboration undersöks sambandet mellan ett antal variabler som beskriver bostadsområden i Kalifornien och det medianbostadsvärde som observerats i respektive område. Datasetet, som härrör från folkräkningen 1990, innehåller 20 640 observationer och beskriver varje censusblock med uppgifter om bland annat medianinkomst, husålder, antal rum, befolkningsstorlek, geografisk position och närheten till havet.

Responsvariabeln i analysen är `median_house_value`, det vill säga medianpriset på bostäder i ett givet block. Syftet är att med hjälp av multipel linjär regression (OLS) skatta hur de tillgängliga variablerna bidrar till att förklara prisvariationen, och att undersöka modellens styrkor och svagheter med statistiska verktyg: signifikanstest, konfidensintervall, Pearson-korrelation och residualdiagnostik.

## Upplägg

Notebooken är strukturerad i tre modellsteg: en full modell för att identifiera multikollinearitet, en förbättrad modell med omkonstruerade särdrag för stabilare koefficienter, samt en log-linjär modell för att bättre uppfylla antaganden om residualvarians och residualfördelning.

## Metod

Modellen implementeras i en egen klass (`LinearRegression` i `linear_regression.py`) som enbart använder numpy för linjär algebra och scipy.stats för fördelningar och test. Inga färdiga regressionsimplementationer eller visualiseringsbibliotek används.

Arbetsgången börjar med att läsa in CSV-filen och konvertera alla värden till numeriska arrayer. Kolumnen `total_bedrooms` innehåller 207 saknade värden, vilka imputeras med kolumnens medelvärde för att bevara stickprovsstorleken. Eftersom datasetet innehåller flera censusblock med identiska koordinater aggregeras sedan observationerna per unik (longitude, latitude)-kombination: additiva variabler (`total_rooms`, `total_bedrooms`, `population`, `households`) summeras, medan nivåvariabler (`housing_median_age`, `median_income`, `median_house_value`) ersätts av medelvärdet inom gruppen. Dessutom tas alla observationer med `median_house_value` = 500 001 bort, eftersom detta värde utgör en artificiell cap som bryter mot OLS-antagandena.

Den kategoriska variabeln `ocean_proximity` one-hot-kodas med referenskategorin `<1H OCEAN` utelämnad (drop-first) för att undvika linjärt beroende med interceptet.

Analysen genomförs i tre steg. Först skattas en fullständig modell med samtliga originalvariabler för att identifiera multikollinearitet och andra problem. Därefter konstrueras en förbättrad modell där de starkt korrelerade blockstorlek-variablerna ersätts med kvotvariabler (`rooms_per_household`, `people_per_household`) och koordinaterna ersätts av ett enda avståndsmått (`distance_to_centroid`). Slutligen angrips det kvarvarande problemet — heteroskedasticitet och icke-normalfördelade residualer — genom en log-linjär modell: responsvariabeln log-transformeras och de numeriska särdragen z-standardiseras. Log-transformationen bygger på insikten att bostadspriser i grunden är multiplikativa — en procentuell felterm är mer realistisk än en absolut — och den gör OLS-antagandena mer rimliga.

Koefficienterna skattas med normalequationen $\hat{\beta} = (X^TX)^{-1}X^Ty$. Implementationen använder `np.linalg.pinv` i stället för en vanlig invers, vilket innebär att beräkningen inte kraschar om $X^TX$ är singulär eller nära-singulär. Det är dock viktigt att förstå att pseudoinversen inte löser de statistiska problem som multikollinearitet medför — den ger enbart en numerisk lösning.

Moore–Penrose-pseudoinversen möjliggör alltså skatten även när designmatrisen är rankbristig, men den reducerar inte den stora variansen som uppstår när prediktorerna nästan är linjärt beroende. Koefficienterna kan fortfarande få höga standardfel, breda konfidensintervall och instabila tecken. Därför är pseudoinversen en beräkningsmässig nödlösning snarare än en statistisk kur. För att hantera multikollinearitet krävs istället åtgärder såsom variabelselektion, regularisering eller dimensionsreduktion som faktiskt förändrar informationsinnehållet i designmatrisen.

Efter skattningen utvärderas varje modell genom F-test och R² för övergripande signifikans, t-test och konfidensintervall för enskilda koefficienter, Pearson-korrelation för att identifiera multikollinearitet, samt residualstatistik för att bedöma modellens antaganden.

In [1]:
import numpy as np
import csv
from scipy.stats import normaltest
from linear_regression import LinearRegression

## 1. Datainläsning och förbehandling

Datasetet läses in från `housing.csv` med Pythons csv-modul. Varje rad representerar ett censusblock. De åtta numeriska kolumnerna konverteras till flyttal, och tomma celler tolkas som saknade värden (NaN). Dessa ersätts med respektive kolumns medelvärde.

Eftersom datasetet innehåller flera block med identiska koordinater (samma longitud och latitud) aggregeras sedan observationerna per unik geografisk punkt: additiva variabler summeras och nivåvariabler medelvärderas. Därefter tas alla observationer bort där `median_house_value` är exakt 500 001, eftersom detta värde representerar en artificiell övre gräns i datasetet. Att behålla dessa censurerade observationer skulle systematiskt snedvrida modellens residualer i det övre prisintervallet.

In [2]:
path = 'housing.csv'
with open(path, newline='', encoding='utf-8') as f:
    r = csv.DictReader(f)
    rows = list(r)
cols_num = [
    'longitude','latitude','housing_median_age','total_rooms','total_bedrooms',
    'population','households','median_income'
]
y_col = 'median_house_value'
cat_col = 'ocean_proximity'

n_raw = len(rows)
X_raw = np.empty((n_raw, len(cols_num)), dtype=float)
y_raw = np.empty(n_raw, dtype=float)
cat_raw = np.empty(n_raw, dtype=object)
for i, row in enumerate(rows):
    for j, c in enumerate(cols_num):
        v = row[c]
        X_raw[i, j] = float(v) if v != '' else np.nan
    y_raw[i] = float(row[y_col])
    cat_raw[i] = row[cat_col]

missing_before = {c: int(np.isnan(X_raw[:, j]).sum()) for j, c in enumerate(cols_num)}

col_means = np.nanmean(X_raw, axis=0)
nan_inds = np.where(np.isnan(X_raw))
X_raw[nan_inds] = np.take(col_means, nan_inds[1])

sum_cols = {'total_rooms', 'total_bedrooms', 'population', 'households'}
mean_cols = {'housing_median_age', 'median_income'}
col_idx = {c: j for j, c in enumerate(cols_num)}

groups = {}
for i in range(n_raw):
    key = (X_raw[i, col_idx['longitude']], X_raw[i, col_idx['latitude']])
    if key not in groups:
        groups[key] = []
    groups[key].append(i)

n_unique = len(groups)
X_agg = np.empty((n_unique, len(cols_num)), dtype=float)
y_agg = np.empty(n_unique, dtype=float)
cat_agg = np.empty(n_unique, dtype=object)

for g_idx, (key, indices) in enumerate(groups.items()):
    idx_arr = np.array(indices)
    for j, c in enumerate(cols_num):
        if c in sum_cols:
            X_agg[g_idx, j] = np.sum(X_raw[idx_arr, j])
        elif c in mean_cols:
            X_agg[g_idx, j] = np.mean(X_raw[idx_arr, j])
        else:
            X_agg[g_idx, j] = X_raw[idx_arr[0], j]
    y_agg[g_idx] = np.mean(y_raw[idx_arr])
    cat_agg[g_idx] = cat_raw[indices[0]]

cap_mask = y_agg < 500001
X_num = X_agg[cap_mask]
y = y_agg[cap_mask]
cat = cat_agg[cap_mask]
n = len(y)

{
    'observationer_ratt': n_raw,
    'saknade_varden_fore_imputering': {k: v for k, v in missing_before.items() if v > 0},
    'unika_koordinater': n_unique,
    'borttagna_cappade (y=500001)': int((~cap_mask).sum()),
    'observationer_efter_rensning': n,
}

{'observationer_ratt': 20640,
 'saknade_varden_fore_imputering': {'total_bedrooms': 207},
 'unika_koordinater': 12590,
 'borttagna_cappade (y=500001)': 525,
 'observationer_efter_rensning': 12065}

## 2. Explorativ dataanalys (EDA)

För att få en överblick av datasetet beräknas deskriptiv statistik (minimum, medelvärde, median, maximum och standardavvikelse) för samtliga numeriska variabler samt responsvariabeln. Dessutom undersöks fördelningen av den kategoriska variabeln `ocean_proximity`.

In [3]:
all_cols = cols_num + [y_col]
all_data = np.column_stack([X_num, y])

descriptive = {}
for j, name in enumerate(all_cols):
    col = all_data[:, j]
    descriptive[name] = {
        'min': round(float(np.min(col)), 2),
        'medel': round(float(np.mean(col)), 2),
        'median': round(float(np.median(col)), 2),
        'max': round(float(np.max(col)), 2),
        'std': round(float(np.std(col)), 2),
    }

ocean_categories, ocean_counts = np.unique(cat.astype(str), return_counts=True)
kategori_fordelning = {str(c): int(cnt) for c, cnt in zip(ocean_categories, ocean_counts)}

{
    'deskriptiv_statistik': descriptive,
    'ocean_proximity_fordelning': kategori_fordelning,
}


{'deskriptiv_statistik': {'longitude': {'min': -124.35,
   'medel': -119.68,
   'median': -119.31,
   'max': -114.31,
   'std': 2.05},
  'latitude': {'min': 32.54,
   'medel': 35.93,
   'median': 35.4,
   'max': 41.95,
   'std': 2.23},
  'housing_median_age': {'min': 1.0,
   'medel': 25.31,
   'median': 25.0,
   'max': 52.0,
   'std': 11.51},
  'total_rooms': {'min': 2.0,
   'medel': 4332.53,
   'median': 3458.0,
   'max': 39320.0,
   'std': 3409.05},
  'total_bedrooms': {'min': 2.0,
   'medel': 891.87,
   'median': 678.0,
   'max': 11493.0,
   'std': 779.42},
  'population': {'min': 5.0,
   'medel': 2374.14,
   'median': 1798.0,
   'max': 35682.0,
   'std': 2067.06},
  'households': {'min': 2.0,
   'medel': 828.39,
   'median': 632.0,
   'max': 10181.0,
   'std': 728.21},
  'median_income': {'min': 0.5,
   'medel': 3.81,
   'median': 3.58,
   'max': 15.0,
   'std': 1.6},
  'median_house_value': {'min': 14999.0,
   'medel': 188085.24,
   'median': 167300.0,
   'max': 500000.9,
   'std'

### Diskussion kring särdragsval

Analysen genomförs i tre steg. I det första steget inkluderas samtliga tillgängliga numeriska variabler samt den kategoriska variabeln `ocean_proximity`. Detta görs medvetet för att kunna identifiera och studera multikollinearitet — fyra av variablerna (`total_rooms`, `total_bedrooms`, `population`, `households`) mäter alla blockstorlek och är därmed starkt korrelerade (Pearson-r mellan 0.86 och 0.97). Att inkludera samtliga gör det möjligt att visa hur korrelationsanalysen avslöjar dessa problem och vilka konsekvenser det får för signifikanstesterna.

Inte alla parametrar är emellertid lämpliga att behålla i en regressionsmodell. Ett särdrag bör ingå endast om det tillför oberoende förklaringsinformation, är teoretiskt motiverat och inte introducerar överdriven multikollinearitet. Att inkludera starkt korrelerade variabler blåser upp koefficienternas varians och gör tolkningen svår. Samtidigt får inga variabler plockas bort enbart av bekvämlighet, eftersom det riskerar att skapa utelämnad variabel-bias. Parameterval måste därför balansera statistiska diagnoser mot domänkunskap om hur bostadspriser faktiskt bildas.

I det andra steget konstrueras en förbättrad modell som adresserar de identifierade problemen. De fyra blockstorlek-variablerna ersätts med två kvotvariabler: `rooms_per_household` och `people_per_household`. Koordinaterna `longitude` och `latitude`, som också uppvisar stark korrelation (r ≈ −0.92), ersätts av ett enda avståndsmått — det euklidiska avståndet till datasetets geografiska centroid. Dessa transformationer minskar kollineariteten avsevärt och gör de kvarvarande koefficienterna stabilare och mer tolkbara.

I det tredje steget log-transformeras responsvariabeln för att bättre matcha antagandet om jämn residualvarians och för att minska skevhet i feltermerna. Samtidigt z-standardiseras numeriska särdrag så att koefficienterna blir direkt jämförbara.

Variabeln `ocean_proximity` one-hot-kodas med referenskategorin `<1H OCEAN` utelämnad (drop-first), vilket är nödvändigt för att undvika att dummy-variablerna bildar en linjärkombination med interceptet.

## 3. Modellskattning

Designmatrisen konstrueras genom att sammanfoga de åtta numeriska kolumnerna med de fyra dummy-variablerna från one-hot-kodningen (fem kategorier minus en referenskategori). Klassen lägger automatiskt till en kolumn med ettor för interceptet, vilket ger totalt 13 kolumner i designmatrisen och 12 skattade särdragskoefficienter utöver interceptet.

In [4]:
model = LinearRegression(confidence_level=0.95, add_intercept=True, drop_first_category=True)
X_cat, categories = model.one_hot_encode(cat, drop_first=model.drop_first_category)
X = np.column_stack([X_num, X_cat])
feature_names = cols_num + [f'{cat_col}={c}' for c in categories]
model.fit(X, y, feature_names=feature_names)

{
    'feature_names': feature_names,
    'antal_sardrag (d)': model.d,
    'antal_observationer (n)': model.n,
}

{'feature_names': ['longitude',
  'latitude',
  'housing_median_age',
  'total_rooms',
  'total_bedrooms',
  'population',
  'households',
  'median_income',
  'ocean_proximity=INLAND',
  'ocean_proximity=ISLAND',
  'ocean_proximity=NEAR BAY',
  'ocean_proximity=NEAR OCEAN'],
 'antal_sardrag (d)': 12,
 'antal_observationer (n)': 12065}

## 4. Felmetrik

De grundläggande felmåtten ger en uppfattning om hur stora avvikelser modellen producerar. Variansen $s^2$ är en väntevärdesriktig skattning av feltermens varians (med $n - d - 1$ i nämnaren), standardavvikelsen $s$ är dess kvadratrot, och RMSE beräknas som kvadratroten ur det genomsnittliga kvadratfelet. RMSE och $s$ ligger nära varandra här eftersom $n$ är stort relativt $d$.

In [5]:
{
    'varians (s²)': model.variance(),
    'standardavvikelse (s)': model.standard_deviation(),
    'RMSE': model.rmse(),
}

{'varians (s²)': 3157411729.493205,
 'standardavvikelse (s)': 56190.85094117373,
 'RMSE': 56160.57004846399}

## 5. Övergripande modellrelevans och signifikans

F-testet prövar nollhypotesen att samtliga koefficienter (utom interceptet) är noll, det vill säga att ingen av de inkluderade variablerna bidrar till att förklara responsvariabeln. R² anger andelen av den totala variansen i Y som förklaras av regressionen. Justerat R² korrigerar för antalet parametrar och ger en mer rättvisande bild vid många särdrag, men i detta fall med $n \approx 12\,000$ och $d = 12$ är skillnaden försumbar.

In [6]:
{
    'R²': model.r2(),
    'justerat_R²': model.adjusted_r2(),
    'F-test': model.f_test(),
}

{'R²': 0.6857868302657955,
 'justerat_R²': 0.6854739728116956,
 'F-test': {'f_stat': 2192.0105187811623,
  'df1': 12,
  'df2': 12052,
  'p_value': 0.0}}

## 6. Individuella koefficienttest och konfidensintervall

Tabellen nedan redovisar varje koefficients skattade värde, standardfel, t-statistika, p-värde och 95-procentigt konfidensintervall.

Ett 95-procentigt konfidensintervall är den etablerade standarden i regressionsanalys och fungerar som en praktisk kompromiss mellan statistisk säkerhet och intervallprecision. En högre nivå, exempelvis 99%, skulle minska risken för typ-I-fel men samtidigt göra intervallen bredare och parameterinterpretationen mindre skarp. I detta fall räcker 95% för att uppnå hög tillförlitlighet, eftersom standardfelen redan är små tack vare stickprovsstorleken.

I stora stickprov blir statistisk signifikans lätt att uppnå även för mycket små effekter — standardfelen krymper med $1/\sqrt{n}$ — vilket innebär att nästan alla koefficienter förblir signifikanta även vid 99% konfidensnivå. Därför måste varje intervall tolkas tillsammans med koefficientens effektstorlek och den praktiska relevansen på bostadsmarknaden, inte enbart genom att konstatera om noll ligger utanför intervallet.

Det är också avgörande att beakta multikollineariteten. T-testet för en enskild koefficient prövar om variabeln tillför förklaringskraft givet att alla andra prediktorer redan finns i modellen. När flera variabler mäter i stort sett samma fenomen (t.ex. `total_rooms`, `total_bedrooms`, `households` och `population`) delar de på informationen, vilket blåser upp standardfelen, breddar konfidensintervallen och gör t-testerna mindre tillförlitliga. En variabel kan då framstå som icke-signifikant, inte för att sambandet saknas, utan för att en närliggande variabel redan fångar samma variation. De fyra blockstorleksvariablerna bör därför tolkas som en grupp snarare än isolerat.

En 95-procentig konfidensnivå är därmed lämplig eftersom den balanserar statistisk tillförlitlighet mot intervallprecision. En ännu högre nivå, såsom 99%, skulle visserligen reducera risken för typ-I-fel men producera bredare intervall och sämre tolkbarhet. Slutsatsen måste därför bygga på en samtidig värdering av (i) om noll ligger utanför intervallet, (ii) hur stor effekten är i praktiska termer och (iii) om sambandet är teoretiskt rimligt.

In [7]:
model.summary()

============================= REGRESSION RESULTS =============================
Observations: 12065           R-squared:      0.6858
Features:     12              Adj. R-squared: 0.6855
RMSE:         56160.5700      F-statistic:    2192.0105
Res. Std Err: 56190.8509      Prob (F-stat):  0
------------------------------------------------------------------------------
                                   Coef    Std Err        t    P>|t| [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept *                  -1642064.4854 80856.4438   -20.31   0.0000 -1800556.1202 -1483572.8507
longitude *                  -19314.8318   927.9813   -20.81   0.0000 -21133.8243 -17495.8393
latitude *                   -17797.8615   900.3060   -19.77   0.0000 -19562.6060 -16033.1169
housing_median_age *           746.7315    48.6491    15.35   0.0000 651.3715 842.0915
total_rooms *                   -1.9464     0.4991    -3.90   0.0000  -2.9246  -0.9681
tot

## 7. Beroendeanalys (Pearson-korrelation)

Pearson-korrelationskoefficienten mäter styrkan i det linjära sambandet mellan två variabler. Värden nära ±1 innebär att variablerna i praktiken bär samma information, vilket är problematiskt i en regressionsmodell: om två prediktorer är nästan linjärkombinationer av varandra kan OLS inte på ett meningsfullt sätt avgöra vilken av dem som "orsakar" variationen i Y. Nedan beräknas korrelationsmatrisen för samtliga särdrag i designmatrisen, exklusive interceptet.

In [8]:
pearson = model.pearson_pairs(X, include_intercept=False)
r_matrix = pearson['r']

n_feats = len(feature_names)
starka_korrelationer = {}
for i in range(n_feats):
    for j in range(i + 1, n_feats):
        if abs(r_matrix[i, j]) > 0.8:
            starka_korrelationer[f'{feature_names[i]} <-> {feature_names[j]}'] = float(np.round(r_matrix[i, j], 3))

{
    'korrelationsmatris': np.round(r_matrix, 2).tolist(),
    'legend': {i: name for i, name in enumerate(feature_names)},
    'starka_korrelationer (|r| > 0.8)': starka_korrelationer,
}


{'korrelationsmatris': [[1.0,
   -0.91,
   -0.1,
   0.11,
   0.12,
   0.15,
   0.11,
   0.03,
   -0.03,
   0.01,
   -0.37,
   0.02],
  [-0.91,
   1.0,
   0.01,
   -0.19,
   -0.2,
   -0.24,
   -0.2,
   -0.15,
   0.34,
   -0.02,
   0.25,
   -0.16],
  [-0.1, 0.01, 1.0, -0.03, 0.05, 0.06, 0.07, -0.19, -0.19, 0.03, 0.19, 0.05],
  [0.11, -0.19, -0.03, 1.0, 0.93, 0.86, 0.93, 0.16, -0.24, -0.02, 0.11, 0.02],
  [0.12, -0.2, 0.05, 0.93, 1.0, 0.9, 0.99, -0.03, -0.25, -0.01, 0.11, 0.03],
  [0.15, -0.24, 0.06, 0.86, 0.9, 1.0, 0.92, -0.02, -0.27, -0.02, 0.06, 0.01],
  [0.11, -0.2, 0.07, 0.93, 0.99, 0.92, 1.0, -0.01, -0.28, -0.02, 0.11, 0.03],
  [0.03,
   -0.15,
   -0.19,
   0.16,
   -0.03,
   -0.02,
   -0.01,
   1.0,
   -0.32,
   -0.01,
   0.1,
   0.03],
  [-0.03,
   0.34,
   -0.19,
   -0.24,
   -0.25,
   -0.27,
   -0.28,
   -0.32,
   1.0,
   -0.02,
   -0.26,
   -0.32],
  [0.01,
   -0.02,
   0.03,
   -0.02,
   -0.01,
   -0.02,
   -0.02,
   -0.01,
   -0.02,
   1.0,
   -0.01,
   -0.01],
  [-0.37, 0.25

### Tolkning av korrelationsmatrisen

Den mest påfallande strukturen i korrelationsmatrisen är det starka linjära beroendet mellan de fyra blockstorlek-variablerna: `total_rooms`, `total_bedrooms`, `households` och `population` uppvisar parvis Pearson-r mellan 0.86 och 0.97. Det extremaste paret är `total_bedrooms` och `households` med r = 0.975, vilket innebär att ungefär 95% av variansen i den ena variabeln redan förklaras av den andra. Att inkludera båda i samma regression ger därför mycket lite extra information, men kostar i form av uppblåsta standardfel och instabila koefficienter.

Konsekvensen för signifikanstesterna är konkret: t-testet för en enskild koefficient mäter om variabeln tillför förklaringskraft utöver det som de övriga variablerna redan fångar. När två variabler mäter i princip samma sak delar de på denna förklaringskraft, och båda kan få onödigt höga standardfel. I värsta fall kan en genuint viktig variabel framstå som icke-signifikant, enbart för att en snarlikt korrelerad variabel redan finns i modellen. Koefficienternas storlek och till och med tecken kan dessutom ändras drastiskt om en av de korrelerade variablerna tas bort — ett tydligt tecken på att de individuella skattningarna inte är stabila. I denna modell har de fyra blockstorlek-variablerna ändå samtliga signifikanta p-värden, men det beror sannolikt på det mycket stora stickprovet snarare än på att modellen kan separera deras individuella bidrag.

Severe multikollinearitet snedvrider inte OLS-koefficienterna i sig, men den blåser upp deras varians (motsvarande höga variance inflation factors), vilket leder till tre följder: (1) standardfelen växer och t-tester får lägre styrka, (2) konfidensintervallen blir bredare och mindre informativa, och (3) koefficienternas tecken kan bli instabila om modellen specificeras om. Den centrala problemlinjen är alltså: multikollinearitet → högre koefficientvarians → större standardfel → bredare konfidensintervall → svagare t-test → sämre tolkbarhet.

Koordinaterna `longitude` och `latitude` har r ≈ −0.92, vilket är förväntat givet Kaliforniens geografi (kusten löper ungefär nordväst–sydost). Även här bör koefficienterna tolkas tillsammans snarare än var för sig.

Variabeln `median_income` uppvisar däremot låg korrelation med samtliga övriga prediktorer (|r| < 0.25) och är därmed det mest oberoende särdraget i modellen. Dess koefficient och signifikanstest är följaktligen de mest tillförlitliga.

## 8. Jämförelse av konfidensnivåer (95% vs 99%)

Som diskuterades ovan är valet av konfidensnivå inte trivialt vid stora stickprov. Nedan visas samma modell med 99-procentiga konfidensintervall. Intervallen breddas — det kritiska t-värdet ökar från ca 1.96 till ca 2.58 — men i denna modell förblir samtliga koefficienter signifikanta vid 99% också. Att resultaten knappt förändras illustrerar att med cirka 12 000 observationer har testerna mycket hög statistisk kraft och att valet mellan 95% och 99% i detta fall inte ger kvalitativt annorlunda slutsatser. En mer meningsfull distinktion vore att jämföra konfidensintervallens bredd med koefficienternas praktiska tolkningsbarhet, snarare än att enbart titta på om p-värdet passerar en viss tröskel. I avsnitt 12 prövas därför en konfidensnivå anpassad till modellens faktiska förklaringsgrad.

I stora stickprov innebär statistisk signifikans inte automatiskt att effekten är praktiskt viktig. Mycket små effekter kan bli signifikanta när standardfelen krymper, och man riskerar då att övertolka obetydliga förändringar. Därför måste konfidensintervall alltid värderas både ur statistiskt (om noll exkluderas) och praktiskt (om intervallet rymmer en meningsfull effektstorlek) perspektiv.

In [9]:
model_99 = LinearRegression(confidence_level=0.99, add_intercept=True, drop_first_category=True)
model_99.fit(X, y, feature_names=feature_names)

model_99.summary()

============================= REGRESSION RESULTS =============================
Observations: 12065           R-squared:      0.6858
Features:     12              Adj. R-squared: 0.6855
RMSE:         56160.5700      F-statistic:    2192.0105
Res. Std Err: 56190.8509      Prob (F-stat):  0
------------------------------------------------------------------------------
                                   Coef    Std Err        t    P>|t| [99.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept *                  -1642064.4854 80856.4438   -20.31   0.0000 -1850369.8726 -1433759.0983
longitude *                  -19314.8318   927.9813   -20.81   0.0000 -21705.5317 -16924.1318
latitude *                   -17797.8615   900.3060   -19.77   0.0000 -20117.2633 -15478.4596
housing_median_age *           746.7315    48.6491    15.35   0.0000 621.3999 872.0631
total_rooms *                   -1.9464     0.4991    -3.90   0.0000  -3.2321  -0.6607
tot

## 9. Predikterade vs. Faktiska värden

För att bedöma modellens träffsäkerhet jämförs de predikterade värdena med de faktiska. Nedan delas observationerna upp i prisintervall, och för varje intervall redovisas medelresidual och residualernas standardavvikelse. Om modellen vore perfekt skulle medelresidualen vara noll i varje intervall. Avvikelser visar i vilka prisklasser modellen systematiskt över- eller underskattar.

I stora stickprov innebär statistisk signifikans inte automatiskt att effekten är praktiskt viktig. Mycket små effekter kan bli statistiskt signifikanta när standardfelen krymper. Därför måste konfidensintervallen bedömas med både statistisk och praktisk betydelse i åtanke.

In [10]:
y_pred = model.predict(X)
residuals = model.residuals()

bins = [0, 100000, 200000, 300000, 400000, 500001, np.inf]
labels = ['0-100k', '100k-200k', '200k-300k', '300k-400k', '400k-500k', '500k+']
pred_vs_actual = {}
for k in range(len(bins) - 1):
    mask = (y >= bins[k]) & (y < bins[k + 1])
    if mask.sum() > 0:
        errors = residuals[mask]
        pred_vs_actual[labels[k]] = {
            'antal': int(mask.sum()),
            'medel_residual': round(float(np.mean(errors)), 2),
            'std_residual': round(float(np.std(errors)), 2),
        }

{
    'prediktionssammanfattning': {
        'medel_predikterat': round(float(np.mean(y_pred)), 2),
        'medel_faktiskt': round(float(np.mean(y)), 2),
        'korrelation_pred_vs_faktiskt': round(float(np.corrcoef(y, y_pred)[0, 1]), 4),
    },
    'residualer_per_prisintervall': pred_vs_actual,
}

{'prediktionssammanfattning': {'medel_predikterat': 188085.24,
  'medel_faktiskt': 188085.24,
  'korrelation_pred_vs_faktiskt': 0.8281},
 'residualer_per_prisintervall': {'0-100k': {'antal': 2560,
   'medel_residual': -22130.56,
   'std_residual': 34396.23},
  '100k-200k': {'antal': 4890,
   'medel_residual': -17103.12,
   'std_residual': 43789.57},
  '200k-300k': {'antal': 2870,
   'medel_residual': 4182.71,
   'std_residual': 44409.02},
  '300k-400k': {'antal': 1222,
   'medel_residual': 56032.73,
   'std_residual': 54989.08},
  '400k-500k': {'antal': 523,
   'medel_residual': 114363.54,
   'std_residual': 74221.11}}}

## 10. Residualdiagnostik

OLS bygger på antagandet att feltermerna är oberoende, har konstant varians (homoskedasticitet) och är approximativt normalfördelade. Om dessa antaganden inte håller kan de beräknade standardfelen och därmed alla signifikanstest och konfidensintervall bli missvisande. Nedan undersöks residualernas fördelning numeriskt: kvartiler, skevhet, kurtosis, ett formellt normalitetstest (D'Agostino–Pearson) samt en enkel jämförelse av residualspridningen vid låga respektive höga predikterade värden som indikation på heteroskedasticitet.

In [11]:
residuals = model.residuals()

quartiles = np.percentile(residuals, [0, 25, 50, 75, 100])

_, normaltest_p = normaltest(residuals)

low_pred = y_pred < np.median(y_pred)
high_pred = ~low_pred
std_low = float(np.std(residuals[low_pred]))
std_high = float(np.std(residuals[high_pred]))

{
    'residualer': {
        'medelvarde': round(float(np.mean(residuals)), 4),
        'std': round(float(np.std(residuals)), 2),
        'min': round(float(quartiles[0]), 2),
        'Q1': round(float(quartiles[1]), 2),
        'median': round(float(quartiles[2]), 2),
        'Q3': round(float(quartiles[3]), 2),
        'max': round(float(quartiles[4]), 2),
        'skevhet': round(float(np.mean((residuals - np.mean(residuals))**3) / np.std(residuals)**3), 4),
        'kurtosis': round(float(np.mean((residuals - np.mean(residuals))**4) / np.std(residuals)**4 - 3), 4),
    },
    'normalitetstest (DAgostino-Pearson)': {
        'p_varde': float(normaltest_p),
        'normalfordelat': bool(normaltest_p > 0.05),
    },
    'heteroskedasticitet_indikation': {
        'std_laga_prediktioner': round(std_low, 2),
        'std_hoga_prediktioner': round(std_high, 2),
        'kvot': round(std_high / std_low, 3),
    },
}

{'residualer': {'medelvarde': 0.0,
  'std': 56160.57,
  'min': -542916.15,
  'Q1': -34661.1,
  'median': -7393.26,
  'Q3': 25877.66,
  'max': 395767.12,
  'skevhet': 0.9684,
  'kurtosis': 4.1144},
 'normalitetstest (DAgostino-Pearson)': {'p_varde': 0.0,
  'normalfordelat': False},
 'heteroskedasticitet_indikation': {'std_laga_prediktioner': 47364.76,
  'std_hoga_prediktioner': 63753.01,
  'kvot': 1.346}}

## 11. Sammanfattning av den första modellen

Den första modellen, med samtliga originalvariabler, ger en förklaringsgrad på R² ≈ 0.686 och ett F-test med p ≈ 0, vilket innebär att modellen som helhet är starkt signifikant. Variabeln `median_income` dominerar med t ≈ 94 och är den mest stabila koefficienten tack vare sin låga korrelation med övriga prediktorer.

Korrelationsanalysen avslöjar dock allvarlig multikollinearitet: de fyra blockstorlek-variablerna (`total_rooms`, `total_bedrooms`, `population`, `households`) har parvis Pearson-r mellan 0.86 och 0.99, och koordinaterna `longitude`/`latitude` har r ≈ −0.91. Detta innebär att de individuella koefficienterna för dessa variabler inte är stabila — deras storlek och tecken kan ändras om modellspecifikationen ändras. Att samtliga ändå uppvisar signifikanta p-värden beror sannolikt på det stora stickprovet snarare än på att modellen kan separera deras individuella bidrag.

Residualanalysen visar heteroskedasticitet (kvot hög/låg std ≈ 1.35) och icke-normalfördelade residualer (skevhet ≈ 0.97, excess kurtosis ≈ 4.1). Dessa avvikelser innebär att standardfel, p-värden och konfidensintervall inte fullt ut kan litas på.

Sammantaget motiverar dessa insikter en förbättrad modellspecifikation, vilken presenteras i nästa avsnitt.

## 12. Förbättrad modell — reducerad multikollinearitet

För att adressera de problem som identifierades i den första modellen konstrueras en ny särdragsuppsättning. De fyra blockstorlek-variablerna (`total_rooms`, `total_bedrooms`, `population`, `households`) ersätts med två kvotvariabler: `rooms_per_household` (antal rum per hushåll) och `people_per_household` (antal personer per hushåll). Dessa fångar samma strukturella information men är betydligt mindre korrelerade med varandra. Variabeln `total_bedrooms` tas bort helt eftersom dess effekt i princip redan fångas av de nya kvotvariablerna, och den var dessutom den enda kolumnen med saknade värden.

Koordinaterna `longitude` och `latitude`, som uppvisade r ≈ −0.91, ersätts av ett enda mått: `distance_to_centroid`, det euklidiska avståndet från varje punkt till datasetets geografiska medelpunkt. Detta komprimerar den geografiska informationen till en enda dimension utan att förlora den huvudsakliga variationen — avstånd från centrum av Kaliforniens bostadsbestånd.

In [12]:
lon_idx = col_idx['longitude']
lat_idx = col_idx['latitude']
centroid_lon = np.mean(X_num[:, lon_idx])
centroid_lat = np.mean(X_num[:, lat_idx])
distance_to_centroid = np.sqrt(
    (X_num[:, lon_idx] - centroid_lon)**2 + (X_num[:, lat_idx] - centroid_lat)**2
)

rooms_per_household = X_num[:, col_idx['total_rooms']] / X_num[:, col_idx['households']]
people_per_household = X_num[:, col_idx['population']] / X_num[:, col_idx['households']]

X_num_2 = np.column_stack([
    distance_to_centroid,
    X_num[:, col_idx['housing_median_age']],
    X_num[:, col_idx['median_income']],
    rooms_per_household,
    people_per_household,
])

feature_names_2 = [
    'distance_to_centroid',
    'housing_median_age',
    'median_income',
    'rooms_per_household',
    'people_per_household',
]

X_cat_2, categories_2 = LinearRegression.one_hot_encode(cat, drop_first=True)
X2 = np.column_stack([X_num_2, X_cat_2])
feature_names_2 = feature_names_2 + [f'{cat_col}={c}' for c in categories_2]

model_2 = LinearRegression(confidence_level=0.95, add_intercept=True, drop_first_category=True)
model_2.fit(X2, y, feature_names=feature_names_2)

{
    'feature_names': feature_names_2,
    'antal_sardrag (d)': model_2.d,
    'antal_observationer (n)': model_2.n,
}

{'feature_names': ['distance_to_centroid',
  'housing_median_age',
  'median_income',
  'rooms_per_household',
  'people_per_household',
  'ocean_proximity=INLAND',
  'ocean_proximity=ISLAND',
  'ocean_proximity=NEAR BAY',
  'ocean_proximity=NEAR OCEAN'],
 'antal_sardrag (d)': 9,
 'antal_observationer (n)': 12065}

In [13]:
model_2.summary()

============================= REGRESSION RESULTS =============================
Observations: 12065           R-squared:      0.6435
Features:     9               Adj. R-squared: 0.6433
RMSE:         59818.1433      F-statistic:    2418.0425
Res. Std Err: 59842.9486      Prob (F-stat):  0
------------------------------------------------------------------------------
                                   Coef    Std Err        t    P>|t| [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept *                  55452.9497  3115.5322    17.80   0.0000 49346.0056 61559.8937
distance_to_centroid *       -4700.7219   519.8062    -9.04   0.0000 -5719.6256 -3681.8183
housing_median_age *           812.8372    51.2368    15.86   0.0000 712.4048 913.2696
median_income *              38651.7574   393.2437    98.29   0.0000 37880.9366 39422.5782
rooms_per_household .          572.0198   214.3836     2.67   0.0076 151.7934 992.2462
people_per_househo

### Pearson-korrelation i den förbättrade modellen

För att verifiera att de nya variablerna verkligen minskat multikollineariteten beräknas Pearson-korrelationsmatrisen på nytt. Målet är att inga par av prediktorer ska ha |r| > 0.8.

In [14]:
pearson_2 = model_2.pearson_pairs(X2, include_intercept=False)
r_matrix_2 = pearson_2['r']

n_feats_2 = len(feature_names_2)
starka_korr_2 = {}
for i in range(n_feats_2):
    for j in range(i + 1, n_feats_2):
        if abs(r_matrix_2[i, j]) > 0.8:
            starka_korr_2[f'{feature_names_2[i]} <-> {feature_names_2[j]}'] = float(np.round(r_matrix_2[i, j], 3))

{
    'korrelationsmatris': np.round(r_matrix_2, 2).tolist(),
    'legend': {i: name for i, name in enumerate(feature_names_2)},
    'starka_korrelationer (|r| > 0.8)': starka_korr_2 if starka_korr_2 else 'Inga — multikollineariteten är löst',
}


{'korrelationsmatris': [[1.0, -0.05, -0.03, 0.06, -0.0, -0.08, 0.0, 0.1, 0.15],
  [-0.05, 1.0, -0.19, -0.17, 0.02, -0.19, 0.03, 0.19, 0.05],
  [-0.03, -0.19, 1.0, 0.24, 0.03, -0.32, -0.01, 0.1, 0.03],
  [0.06, -0.17, 0.24, 1.0, -0.01, 0.14, -0.0, -0.03, -0.05],
  [-0.0, 0.02, 0.03, -0.01, 1.0, 0.01, -0.0, -0.01, -0.0],
  [-0.08, -0.19, -0.32, 0.14, 0.01, 1.0, -0.02, -0.26, -0.32],
  [0.0, 0.03, -0.01, -0.0, -0.0, -0.02, 1.0, -0.01, -0.01],
  [0.1, 0.19, 0.1, -0.03, -0.01, -0.26, -0.01, 1.0, -0.11],
  [0.15, 0.05, 0.03, -0.05, -0.0, -0.32, -0.01, -0.11, 1.0]],
 'legend': {0: 'distance_to_centroid',
  1: 'housing_median_age',
  2: 'median_income',
  3: 'rooms_per_household',
  4: 'people_per_household',
  5: 'ocean_proximity=INLAND',
  6: 'ocean_proximity=ISLAND',
  7: 'ocean_proximity=NEAR BAY',
  8: 'ocean_proximity=NEAR OCEAN'},
 'starka_korrelationer (|r| > 0.8)': 'Inga — multikollineariteten är löst'}

### Konfidensintervall med anpassad konfidensnivå

I denna rapport används 95% som huvudnivå för inferens, eftersom det är den etablerade standarden i regressionsanalys och därför mest lämplig för formella slutsatser om parametrarna. Som robusthetskontroll visas även 99% konfidensintervall; att slutsatserna i stort sett är oförändrade visar att resultaten inte är känsliga för ett striktare signifikanskrav. Den 60-procentiga nivån används endast som ett kompletterande, explorativt perspektiv i den förbättrade modellen för att illustrera intervallbredd och praktisk osäkerhet vid modellens aktuella förklaringsgrad (R² ≈ 0.64). Den ersätter alltså inte 95%-analysen, utan fungerar som en pedagogisk jämförelse.


In [15]:
model_2_60 = LinearRegression(confidence_level=0.60, add_intercept=True, drop_first_category=True)
model_2_60.fit(X2, y, feature_names=feature_names_2)

ci_60 = model_2_60.confidence_intervals()
coef_names_60 = ['intercept'] + feature_names_2
ci_table = {}
for i, name in enumerate(coef_names_60):
    ci_table[name] = {
        'koefficient': round(float(model_2_60.beta[i]), 2),
        'CI_low': round(float(ci_60['lower'][i]), 2),
        'CI_high': round(float(ci_60['upper'][i]), 2),
    }

{
    'konfidensniva': 0.60,
    'konfidensintervall': ci_table,
    'R2': round(model_2_60.r2(), 4),
}

{'konfidensniva': 0.6,
 'konfidensintervall': {'intercept': {'koefficient': 55452.95,
   'CI_low': 52830.76,
   'CI_high': 58075.14},
  'distance_to_centroid': {'koefficient': -4700.72,
   'CI_low': -5138.22,
   'CI_high': -4263.23},
  'housing_median_age': {'koefficient': 812.84,
   'CI_low': 769.71,
   'CI_high': 855.96},
  'median_income': {'koefficient': 38651.76,
   'CI_low': 38320.78,
   'CI_high': 38982.73},
  'rooms_per_household': {'koefficient': 572.02,
   'CI_low': 391.58,
   'CI_high': 752.46},
  'people_per_household': {'koefficient': -244.19,
   'CI_low': -278.09,
   'CI_high': -210.29},
  'ocean_proximity=INLAND': {'koefficient': -64198.84,
   'CI_low': -65334.62,
   'CI_high': -63063.05},
  'ocean_proximity=ISLAND': {'koefficient': 195388.62,
   'CI_low': 172837.54,
   'CI_high': 217939.7},
  'ocean_proximity=NEAR BAY': {'koefficient': 8904.69,
   'CI_low': 7082.55,
   'CI_high': 10726.82},
  'ocean_proximity=NEAR OCEAN': {'koefficient': 23336.6,
   'CI_low': 21774.75,


## 13. Log-linjär modell — att lösa heteroskedasticiteten vid roten

De två första modellerna delar ett fundamentalt problem som ingen mängd feature engineering kan lösa: OLS på råa dollarpriser antar att feltermen är *additiv* och har *konstant varians*. Men bostadspriser beter sig inte så. Ett fel på 30 000 dollar betyder en katastrof för en bostad värd 50 000 men är marginellt för en värd 400 000. Prisvariationen är i grunden *multiplikativ* — den skalar med prisnivån. Detta är exakt vad heteroskedasticitetskvoten på ≈ 1.4 och den positiva skevheten avslöjar.

Lösningen är att inte modellera $Y$ direkt, utan $\log(Y)$:

$$\log(Y_i) = X_i\beta + \epsilon_i$$

Om $\epsilon_i$ har konstant varians innebär det att felen på den ursprungliga skalan är *procentuella*, inte absoluta — vilket är precis hur bostadsmarknaden fungerar. Förutom att adressera heteroskedasticiteten komprimerar logaritmen den högra svansen i prisfördelningen och gör residualerna mer symmetriska, vilket förbättrar normalitetsantagandet.

Utöver log-transformationen görs ytterligare en förbättring: samtliga numeriska särdrag z-standardiseras (subtrahera medelvärdet, dividera med standardavvikelsen) så att koefficienterna uttrycks i samma enhet — antal standardavvikelsers förändring i $\log(Y)$ — och därmed blir direkt jämförbara i storlek. En koefficient på exempelvis 0.3 betyder att en standardavvikelses ökning i det särdraget är associerad med en 35-procentig ökning i medianpriset ($e^{0.3} \approx 1.35$).

Vid återtransformering till den ursprungliga skalan räcker det inte att bara exponentiera prediktionerna: $E[Y] \neq \exp(E[\log Y])$ på grund av Jensens olikhet. Istället används Duans smearing-estimator, $\hat{E}[Y_i] = \exp(\hat{\log Y}_i) \cdot \frac{1}{n}\sum_j\exp(\hat{e}_j)$, som korrigerar för denna bias. Det är dock viktigt att notera att R² på dollarskalan inte är direkt jämförbart med de linjära modellernas R² — log-modellen optimerar en *annan* förlustfunktion (procentuella fel snarare än absoluta dollarfel). Det relevanta kvalitetsmåttet för log-modellen är dess R² på log-skalan samt residualdiagnostiken.

In [16]:
log_y = np.log(y)

num_features_3 = np.column_stack([
    distance_to_centroid,
    X_num[:, col_idx['housing_median_age']],
    X_num[:, col_idx['median_income']],
    rooms_per_household,
    people_per_household,
])
num_names_3 = [
    'distance_to_centroid',
    'housing_median_age',
    'median_income',
    'rooms_per_household',
    'people_per_household',
]

means_3 = np.mean(num_features_3, axis=0)
stds_3 = np.std(num_features_3, axis=0)
Z_num_3 = (num_features_3 - means_3) / stds_3

X_cat_3, categories_3 = LinearRegression.one_hot_encode(cat, drop_first=True)
X3 = np.column_stack([Z_num_3, X_cat_3])
feature_names_3 = [f'z_{name}' for name in num_names_3] + [f'{cat_col}={c}' for c in categories_3]

model_3 = LinearRegression(confidence_level=0.95, add_intercept=True, drop_first_category=True)
model_3.fit(X3, log_y, feature_names=feature_names_3)

{
    'feature_names': feature_names_3,
    'antal_sardrag (d)': model_3.d,
    'antal_observationer (n)': model_3.n,
    'respons': 'log(median_house_value)',
}

{'feature_names': ['z_distance_to_centroid',
  'z_housing_median_age',
  'z_median_income',
  'z_rooms_per_household',
  'z_people_per_household',
  'ocean_proximity=INLAND',
  'ocean_proximity=ISLAND',
  'ocean_proximity=NEAR BAY',
  'ocean_proximity=NEAR OCEAN'],
 'antal_sardrag (d)': 9,
 'antal_observationer (n)': 12065,
 'respons': 'log(median_house_value)'}

In [17]:
model_3.summary()

============================= REGRESSION RESULTS =============================
Observations: 12065           R-squared:      0.6626
Features:     9               Adj. R-squared: 0.6624
RMSE:         0.3226          F-statistic:    2630.8143
Res. Std Err: 0.3227          Prob (F-stat):  0
------------------------------------------------------------------------------
                                   Coef    Std Err        t    P>|t| [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept *                     12.1849     0.0050  2418.54   0.0000  12.1750  12.1948
z_distance_to_centroid *        -0.0230     0.0030    -7.58   0.0000  -0.0289  -0.0170
z_housing_median_age *           0.0203     0.0032     6.38   0.0000   0.0140   0.0265
z_median_income *                0.3176     0.0034    93.57   0.0000   0.3109   0.3243
z_rooms_per_household .          0.0115     0.0031     3.68   0.0002   0.0054   0.0177
z_people_per_household *      

### Pearson-korrelation och residualdiagnostik för log-modellen

Först verifieras att multikollineariteten förblir låg efter standardiseringen. Därefter genomförs samma residualdiagnostik som för den första modellen, nu på log-skalan. Om log-transformationen har fungerat bör vi se: (1) en heteroskedasticitetskvot nära 1.0 istället för 1.4, (2) markant lägre skevhet, och (3) lägre excess kurtosis. Slutligen återtransformeras prediktionerna till dollar med hjälp av Duans smearing-estimator.

In [18]:
pearson_3 = model_3.pearson_pairs(X3, include_intercept=False)
r_matrix_3 = pearson_3['r']

n_feats_3 = len(feature_names_3)
starka_korr_3 = {}
for i in range(n_feats_3):
    for j in range(i + 1, n_feats_3):
        if abs(r_matrix_3[i, j]) > 0.8:
            starka_korr_3[f'{feature_names_3[i]} <-> {feature_names_3[j]}'] = round(r_matrix_3[i, j], 3)

log_residuals = model_3.residuals()
log_pred = model_3.predict(X3)

log_quartiles = np.percentile(log_residuals, [0, 25, 50, 75, 100])
_, log_normaltest_p = normaltest(log_residuals)

log_low = log_pred < np.median(log_pred)
log_high = ~log_low
log_std_low = float(np.std(log_residuals[log_low]))
log_std_high = float(np.std(log_residuals[log_high]))

log_skew = float(np.mean((log_residuals - np.mean(log_residuals))**3) / np.std(log_residuals)**3)
log_kurt = float(np.mean((log_residuals - np.mean(log_residuals))**4) / np.std(log_residuals)**4 - 3)

residuals_m1 = model.residuals()
y_pred_m1 = model.predict(X)
low_m1 = y_pred_m1 < np.median(y_pred_m1)
skew_m1 = float(np.mean((residuals_m1 - np.mean(residuals_m1))**3) / np.std(residuals_m1)**3)
het_m1 = float(np.std(residuals_m1[~low_m1]) / np.std(residuals_m1[low_m1]))

{
    'starka_korrelationer (|r| > 0.8)': starka_korr_3 if starka_korr_3 else 'Inga',
    'residualdiagnostik_jamforelse': {
        'Modell 1 (linjar)': {
            'skevhet': round(skew_m1, 4),
            'heteroskedasticitet_kvot': round(het_m1, 3),
        },
        'Modell 3 (log-linjar)': {
            'skevhet': round(log_skew, 4),
            'heteroskedasticitet_kvot': round(log_std_high / log_std_low, 3),
        },
        'forbattring_skevhet': f'{abs(skew_m1):.2f} -> {abs(log_skew):.2f} ({(1 - abs(log_skew)/abs(skew_m1))*100:.0f}% reduktion)',
        'forbattring_heteroskedasticitet': f'{het_m1:.3f} -> {log_std_high/log_std_low:.3f} (nara 1.0 = perfekt)',
    },
    'residualdetaljer_log_skala': {
        'medelvarde': round(float(np.mean(log_residuals)), 6),
        'std': round(float(np.std(log_residuals)), 4),
        'min': round(float(log_quartiles[0]), 4),
        'Q1': round(float(log_quartiles[1]), 4),
        'median': round(float(log_quartiles[2]), 4),
        'Q3': round(float(log_quartiles[3]), 4),
        'max': round(float(log_quartiles[4]), 4),
        'kurtosis': round(log_kurt, 4),
    },
    'normalitetstest (DAgostino-Pearson)': {
        'p_varde': float(log_normaltest_p),
        'notering': 'Formellt forkastad men drastiskt forbattrad fordelningsform',
    },
}

{'starka_korrelationer (|r| > 0.8)': 'Inga',
 'residualdiagnostik_jamforelse': {'Modell 1 (linjar)': {'skevhet': 0.9684,
   'heteroskedasticitet_kvot': 1.346},
  'Modell 3 (log-linjar)': {'skevhet': 0.1112,
   'heteroskedasticitet_kvot': 0.82},
  'forbattring_skevhet': '0.97 -> 0.11 (89% reduktion)',
  'forbattring_heteroskedasticitet': '1.346 -> 0.820 (nara 1.0 = perfekt)'},
 'residualdetaljer_log_skala': {'medelvarde': 0.0,
  'std': 0.3226,
  'min': -2.6915,
  'Q1': -0.2087,
  'median': -0.0138,
  'Q3': 0.1935,
  'max': 1.654,
  'kurtosis': 1.6466},
 'normalitetstest (DAgostino-Pearson)': {'p_varde': 2.6033680585979694e-103,
  'notering': 'Formellt forkastad men drastiskt forbattrad fordelningsform'}}

In [19]:
smearing_factor = float(np.mean(np.exp(log_residuals)))
y_pred_log_bt = np.exp(log_pred) * smearing_factor

ss_res_bt = float(np.sum((y - y_pred_log_bt)**2))
ss_tot = float(np.sum((y - np.mean(y))**2))
r2_bt = 1.0 - ss_res_bt / ss_tot
rmse_bt = float(np.sqrt(np.mean((y - y_pred_log_bt)**2)))

rmse_m1 = float(np.sqrt(np.mean((y - y_pred_m1)**2)))
y_pred_m2 = model_2.predict(X2)
rmse_m2 = float(np.sqrt(np.mean((y - y_pred_m2)**2)))

pct_errors = (y - y_pred_log_bt) / y * 100
pct_errors_m1 = (y - y_pred_m1) / y * 100

{
    'duan_smearing_factor': round(smearing_factor, 4),
    'jamforelse': {
        'Modell 1 (alla variabler, linjar)': {
            'R2': round(model.r2(), 4),
            'RMSE_dollar': round(rmse_m1, 0),
            'sardrag': model.d,
            'multikollinearitet': '7 par med |r|>0.8',
            'skevhet': round(skew_m1, 2),
            'hetero_kvot': round(het_m1, 3),
            'median_procentfel': round(float(np.median(np.abs(pct_errors_m1))), 1),
        },
        'Modell 2 (kvotvariabler, linjar)': {
            'R2': round(model_2.r2(), 4),
            'RMSE_dollar': round(rmse_m2, 0),
            'sardrag': model_2.d,
            'multikollinearitet': '0 par',
        },
        'Modell 3 (log-linjar, z-standardiserad)': {
            'R2_log_skala': round(model_3.r2(), 4),
            'R2_dollar_Duan': round(r2_bt, 4),
            'RMSE_dollar': round(rmse_bt, 0),
            'sardrag': model_3.d,
            'multikollinearitet': '0 par',
            'skevhet': round(log_skew, 2),
            'hetero_kvot': round(log_std_high / log_std_low, 3),
            'median_procentfel': round(float(np.median(np.abs(pct_errors))), 1),
        },
    },
    'notering': 'R2 pa dollar-skalan ar INTE rattvisande for log-modellen — den optimerar procentuella fel, inte absoluta. Median procentfel ar det mer relevanta mattet.',
}

{'duan_smearing_factor': 1.0547,
 'jamforelse': {'Modell 1 (alla variabler, linjar)': {'R2': 0.6858,
   'RMSE_dollar': 56161.0,
   'sardrag': 12,
   'multikollinearitet': '7 par med |r|>0.8',
   'skevhet': 0.97,
   'hetero_kvot': 1.346,
   'median_procentfel': 19.0},
  'Modell 2 (kvotvariabler, linjar)': {'R2': 0.6435,
   'RMSE_dollar': 59818.0,
   'sardrag': 9,
   'multikollinearitet': '0 par'},
  'Modell 3 (log-linjar, z-standardiserad)': {'R2_log_skala': 0.6626,
   'R2_dollar_Duan': 0.494,
   'RMSE_dollar': 71268.0,
   'sardrag': 9,
   'multikollinearitet': '0 par',
   'skevhet': 0.11,
   'hetero_kvot': 0.82,
   'median_procentfel': 21.3}},
 'notering': 'R2 pa dollar-skalan ar INTE rattvisande for log-modellen — den optimerar procentuella fel, inte absoluta. Median procentfel ar det mer relevanta mattet.'}

## 14. Sammanfattning och diskussion

Analysen genomfördes i tre steg där varje modell byggde vidare på föregående resultat.

**Modell 1** inkluderade samtliga originalvariabler och visade hög förklaringsgrad (R² ≈ 0.69), men också tydlig multikollinearitet (sju par med |r| > 0.8) samt heteroskedastiska och positivt sneda residualer (skevhet ≈ 0.97, heteroskedasticitetskvot ≈ 1.35). Det gör enskilda koefficienttolkningar mindre stabila.

**Modell 2** ersatte blockstorlek-variabler med kvoter (`rooms_per_household`, `people_per_household`) och ersatte koordinater med `distance_to_centroid`. Detta eliminerade starka linjära beroenden (0 par med |r| > 0.8) och gav mer tolkbara koefficienter, men residualproblemen kvarstod i huvudsak.

**Modell 3** log-transformerade responsen och z-standardiserade numeriska särdrag. Det gav tydligt bättre antagandeuppfyllelse: skevheten minskade från ca 0.97 till 0.11 (cirka 89% reduktion) och heteroskedasticitetskvoten gick från ca 1.35 till 0.82 (närmare 1.0). Därmed blir inferens (standardfel, p-värden och konfidensintervall) mer robust. R² på log-skalan (≈ 0.66) är inte direkt jämförbart med linjära modellers R² på dollarskalan eftersom förlustfunktionerna skiljer sig.

Den statistiska tolkningen måste gå bortom ett binärt fokus på p-värden. I stora datamängder innebär ett litet standardfel att även obetydliga effekter blir signifikanta. Koeficienterna beskriver modellens förväntade förändring i bostadspris givet en marginal förändring i prediktorn, men tolkningen är meningsfull endast om intervallet är tillräckligt smalt och effekten är praktiskt relevant. Konfidensintervallens bredd ska därför jämföras med koefficienternas storlek och med vad som är rimligt på bostadsmarknaden.

Det är dessutom viktigt att skilja mellan korrelation och kausalitet. OLS-modellen identifierar statistiska samband mellan särdrag och bostadspris, men den bevisar inte att ett högre `median_income` *orsakar* ett högre pris. Både inkomster och bostadspriser kan styras av gemensamma socioekonomiska eller geografiska faktorer. För att etablera kausala samband skulle man behöva kontrollerade experiment eller specifika kausala modeller. Resultaten i denna rapport ska därför läsas som prediktiva och associativa.

Multikollinearitetens effektkedja kan sammanfattas som: stark korrelation mellan prediktorer → högre koefficientvarians → större standardfel → bredare konfidensintervall → lägre styrka i t-tester → svagare tolkbarhet av enskilda koefficienter. Det påverkar främst inferensens tillförlitlighet snarare än modellens prediktiva kraft, vilket förklarar varför R² kan vara högt samtidigt som koefficienterna är instabila.

Sammanfattningsvis är Modell 1 stark för prediktion i absoluta dollar, Modell 2 reducerar multikollineariteten och ger tydligare särdragsbidrag, medan Modell 3 ger den mest statistiskt tillförlitliga tolkningen genom bättre uppfyllelse av modellantagandena.”}